In [1]:
import glob
from os.path import dirname, basename
import pandas as pd

In [2]:
data_dir = '../data'
results_dir = '../results'
dataset = 'demo'
sf_inpainting_models = [
    # 'fg_flownet2_deepfillv1',
    # 'fg_liteflownet_deepfillv1',
    # 'fg_pwcnet_deepfillv1',
    # 'fg_flownet2_regionfill',
    # 'fg_liteflownet_regionfill',
    # 'fg_pwcnet_regionfill',
    'sf_deepfillv1',
    'sf_deepfillv2',
    'sf_kernelfill',
    'sf_pconvunet',
    'sf_regionfill'
]

object_stats_dfs = []
for path in glob.glob(f'{data_dir}/interim/{dataset}/ObjectStats/*/raw_object_stats.csv'):
    object_stats_df = pd.read_csv(path)
    object_stats_df['object_sequence_name'] = basename(dirname(path))
    object_stats_dfs.append(object_stats_df)
object_stats_df = pd.concat(object_stats_dfs)

results_dfs = []
for model in sf_inpainting_models:
    for path in glob.glob(f'{results_dir}/inpainting/{model}/Evaluation/*/results.csv'):
        results_df = pd.read_csv(path)
        results_df['model'] = model
        sequence_name = basename(dirname(path))
        results_df['sequence_name'] = sequence_name
        results_df['object_sequence_name'] = sequence_name.split('_')[1]
        results_df['object_id'] = int(sequence_name.split('_')[2])
        results_dfs.append(results_df)
results_df = pd.concat(results_dfs)

df = pd.merge(object_stats_df, results_df, on=['object_sequence_name', 'object_id', 'frame_id'])
df

,frame_id,object_id,object_to_screen_ratio,object_sequence_name,mean_squared_error,peak_signal_noise_ratio,structural_similarity,model,sequence_name
0,0,1,0.110939,breakdance,286.754443,23.555702,0.910212,sf_deepfillv1,india_breakdance_001
1,0,1,0.110939,breakdance,187.466077,25.401577,0.912602,sf_deepfillv2,india_breakdance_001
2,0,1,0.110939,breakdance,69.578865,29.706030,0.943475,sf_kernelfill,india_breakdance_001
3,0,1,0.110939,breakdance,310.131040,23.215351,0.884030,sf_pconvunet,india_breakdance_001
4,0,1,0.110939,breakdance,68.944936,29.745780,0.944042,sf_regionfill,india_breakdance_001
...,...,...,...,...,...,...,...,...,...
1275,45,1,0.044289,tennis,128.975881,27.025719,0.956207,sf_deepfillv1,night-race_tennis_001
1276,45,1,0.044289,tennis,777.783498,19.222216,0.938668,sf_deepfillv2,night-race_tennis_001
1277,45,1,0.044289,tennis,14.312490,36.573652,0.982483,sf_kernelfill,night-race_tennis_001
1278,45,1,0.044289,tennis,72.681681,29.516554,0.948164,sf_pconvunet,night-race_tennis_001


In [3]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['mean_squared_error'] = df['mean_squared_error']
metrics['peak_signal_noise_ratio'] = df['peak_signal_noise_ratio']
metrics['structural_similarity'] = df['structural_similarity']
metrics.groupby('model').mean()

,mean_squared_error,peak_signal_noise_ratio,structural_similarity
model,,,
sf_deepfillv1,181.579858,26.075275,0.945851
sf_deepfillv2,215.795784,26.146618,0.945368
sf_kernelfill,73.649317,30.220348,0.962768
sf_pconvunet,266.029874,24.499504,0.935342
sf_regionfill,69.204386,30.434336,0.963328


In [4]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['mean_squared_error'] = df['mean_squared_error'] * df['object_to_screen_ratio']
metrics['peak_signal_noise_ratio'] = df['peak_signal_noise_ratio'] * df['object_to_screen_ratio']
metrics['structural_similarity'] = df['structural_similarity'] * df['object_to_screen_ratio']
metrics.groupby('model').mean()


,mean_squared_error,peak_signal_noise_ratio,structural_similarity
model,,,
sf_deepfillv1,14.190058,1.767519,0.064961
sf_deepfillv2,12.968624,1.826145,0.065078
sf_kernelfill,5.435289,2.058440,0.066259
sf_pconvunet,20.451154,1.649461,0.064103
sf_regionfill,5.018002,2.077817,0.066304
